In [2]:
from utils import imageUtils
from utils import postProcessing
from utils import MultiDefectDetectionDataset
from utils import evaluation
from utils import visualization
import numpy as np
import os
from chainercv.links import FasterRCNNVGG16
from chainercv.visualizations import vis_bbox
from chainercv.utils import write_image
import chainer
import math
from chainercv import utils
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
#load Data
root = './data/3Types/Data3TypesYminXminYmaxXmax5'
dataset = MultiDefectDetectionDataset(data_dir=root, split='train')
dataset_test = MultiDefectDetectionDataset(data_dir=root, split='test')
bbox_label_names = ('111', 'dot','100')

In [4]:
# DataSet Statistics
print('total number of training images: ', len(dataset))
print('total number of test images: ', len(dataset_test))
print('type of defects: ', bbox_label_names)

total number of training images:  153
total number of test images:  12
type of defects:  ('111', 'dot', '100')


Best performance model list

0.81 : "snapshot_model_510000_lr1_20180924.npz"


In [12]:
# import Evaluator
from chainer.iterators import SerialIterator
from chainer.datasets import SubDataset
from chainercv.datasets import VOCBboxDataset
from chainercv.datasets import voc_bbox_label_names
from chainercv.extensions import DetectionVOCEvaluator
from chainercv.links import SSD300

# Only use subset of dataset so that evaluation finishes quickly.
dataset = dataset_test
# set batch size = 1
it = SerialIterator(dataset, 1, repeat=False, shuffle=False)

# Build the detector
use_gpu = False
proposal_params = {'min_size': 8}
model = FasterRCNNVGG16(n_fg_class=3, pretrained_model='./modelResults/snapshot_model_510000_lr1_20180924.npz', ratios=[ 0.5, 1, 1.5, 2, 2.5, 3, 3.5,4],anchor_scales=[1, 4, 8, 16], min_size=1024, max_size=1024,proposal_creator_params=proposal_params)

if use_gpu:
    chainer.cuda.get_device_from_id(0).use()
    model.to_gpu()

bbox_label_names = ('111', 'dot','100')

evaluator = DetectionVOCEvaluator(it, model,label_names=bbox_label_names)
# result is a dictionary of evaluation scores. Print it and check it.
result = evaluator()
print(result)

In [ ]:
# Test the code of calculate P and R by the defect kinds instead of just using the overall data
dataset_test_short = MultiDefectDetectionDataset(data_dir=root, split='shorttest')
%time recalls, precisions = evaluate_set_by_centroid_kind(model,bbox_label_names, dataset_test_short, threshold=0.05)
print("Average recall ", sum(recalls)/len(recalls))
print("Average precision ", sum(precisions)/len(precisions))

In [6]:
# Plot new size report

In [ ]:
bins = 20
sz_range = (15,130)
recalls, precisions = evaluate_set_by_defect_size(model, dataset_test, num_bins=bins, size_range=sz_range, threshold=0.05)
pr_plot_by_size(recalls, label='recall', num_bins=bins, size_range=sz_range)